In [1]:
%%capture
!pip install -r ../model-archiver/model-store/youtubegoes5g/requirements.txt

In [2]:
from torch import nn
import torch

# Build model with non-linear activation function
class InterruptionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(in_features=29, out_features=200)
        self.layer_2 = nn.Linear(in_features=200, out_features=100)
        self.layer_3 = nn.Linear(in_features=100, out_features=1)
        self.relu = nn.ReLU() # <- add in ReLU activation function
        # Can also put sigmoid in the model
        # This would mean you don't need to use it on the predictions
        # self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Intersperse the ReLU activation function between layers
        return self.layer_3(self.relu(self.layer_2(self.relu(self.layer_1(x)))))

In [3]:
import os
from pickle import load
from sklearn.preprocessing import StandardScaler

model = None

class MyHandler():
    def __init__(self, model):
        self.model = model

    def preprocess(self, data):
        """
        Transform raw input into model input data.
        """
        try:
            # Load scaler
            scaler = StandardScaler()
            scaler = load(open('../model-archiver/model-store/youtubegoes5g/scaler.pkl', 'rb'))

            tensor_list = []
            for item in data:
                item = scaler.transform([item['data']])
                tensor_data = torch.tensor(item, dtype=torch.float32)  # Each instance as a tensor
                tensor_list.append(tensor_data)
            # Stack all tensors along a new dimension to create a single tensor
            combined_tensor = torch.cat(tensor_list, dim=0)
            return combined_tensor
        except Exception as e:
            raise ValueError("Failed to preprocess input data: ", e, "data received: ", data)

    def inference(self, model_input):
        """
        Perform model inference.
        """
        try:
            inference_list = []
            for tensor_data in model_input:
                with torch.no_grad():
                    output = torch.round(torch.sigmoid(self.model(tensor_data))).squeeze()
                #inference = output.cpu().numpy().tolist()
                inference_list.append(output)
            return inference_list
        except Exception as e:
            raise RuntimeError("Inference failed:", e)

    def postprocess(self, inference_output):
        """
        Convert model output to a list of predictions.
        """
        try:
            # Process each item in the batch
            result_list = []
            for result in inference_output:
                if result > 0:
                    result_list.append("Stall")
                else:
                    result_list.append("No Stall")
            return result_list
        except Exception as e:
            raise ValueError("Failed to postprocess output data: ", e)

    def handle(self, data):
        """
        Handle a prediction request.
        """
        try:
            model_input = self.preprocess(data)
            model_output = self.inference(model_input)
            return self.postprocess(model_output)
        except Exception as e:
            return [str(e)]

In [4]:
# Instantiate model and handler
serialized_file = "../model-archiver/model-store/youtubegoes5g/model.pt"
    
model = InterruptionModel()
model.load_state_dict(torch.load(serialized_file, weights_only=True))
model.eval()
handler = MyHandler(model)

In [70]:
%%capture
import pandas as pd
import random

# Load the CSV file
csv_file = "dataset.csv"  # Replace with your file path
df = pd.read_csv(csv_file)

# Drop the first 4 columns
df = df.iloc[:, 4:]

# Shuffle the DataFrame to randomize row order
df_shuffled = df.sample(frac=1, random_state=None).reset_index(drop=True)

# Number of rows to extract (should not exceed total rows)
num_iterations = min(512, len(df_shuffled))  # Adjust as needed

# Loop to get unique rows and store them in separate variables
for i in range(1, num_iterations + 1):
    # Select row i (guaranteed to be unique due to shuffling)
    row_values = df_shuffled.iloc[i - 1].values.tolist()
    
    # Dynamically create a variable name and assign the values
    globals()[f"sample_{i}"] = row_values
    
    # Print the selected sample
    print(f"Sample {i}: {globals()[f'sample_{i}']}")
    
# Select a random variable
#random_variable_name = random.choice([f"sample_{i}" for i in range(1, num_iterations + 1)])
#random_variable_value = globals()[random_variable_name]
#data = [{"data": random_variable_value}]

In [75]:
%%timeit -r 10 -n 100
# Single input test
data = [{"data": [13,13,13,0,13,13,13,13,-76,-76,-81,-76,-78.5,-76,-76,-7,-7,-12,-7,-9.5,-7,-7,12,12,7,12,9.5,12,12]}]
#data = [{"data": globals()['sample_1']}]

result = handler.handle(data)

#print("result: " ,result)

3.5 ms ± 451 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [76]:
%%timeit -r 10 -n 100
num_iterations = 4
for i in range(1, num_iterations + 1):
    # Select row i (guaranteed to be unique due to shuffling)
    #data = [{"data": globals()[f'sample_{i}']}]
    data = [{"data": [13,13,13,0,13,13,13,13,-76,-76,-81,-76,-78.5,-76,-76,-7,-7,-12,-7,-9.5,-7,-7,12,12,7,12,9.5,12,12]}]
    result = handler.handle(data)
    #print("result: " ,result)

15.9 ms ± 2.63 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [77]:
%%timeit -r 10 -n 100
num_iterations = 8
for i in range(1, num_iterations + 1):
    # Select row i (guaranteed to be unique due to shuffling)
    #data = [{"data": globals()[f'sample_{i}']}]
    data = [{"data": [13,13,13,0,13,13,13,13,-76,-76,-81,-76,-78.5,-76,-76,-7,-7,-12,-7,-9.5,-7,-7,12,12,7,12,9.5,12,12]}]
    result = handler.handle(data)
    #print("result: " ,result)

30.4 ms ± 4.03 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [78]:
%%timeit -r 10 -n 100
num_iterations = 16
for i in range(1, num_iterations + 1):
    # Select row i (guaranteed to be unique due to shuffling)
    #data = [{"data": globals()[f'sample_{i}']}]
    data = [{"data": [13,13,13,0,13,13,13,13,-76,-76,-81,-76,-78.5,-76,-76,-7,-7,-12,-7,-9.5,-7,-7,12,12,7,12,9.5,12,12]}]
    result = handler.handle(data)
    #print("result: " ,result)

59.5 ms ± 4.07 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [79]:
%%timeit -r 10 -n 100
num_iterations = 32
for i in range(1, num_iterations + 1):
    # Select row i (guaranteed to be unique due to shuffling)
    #data = [{"data": globals()[f'sample_{i}']}]
    data = [{"data": [13,13,13,0,13,13,13,13,-76,-76,-81,-76,-78.5,-76,-76,-7,-7,-12,-7,-9.5,-7,-7,12,12,7,12,9.5,12,12]}]
    result = handler.handle(data)
    #print("result: " ,result)

118 ms ± 5.2 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [80]:
%%timeit -r 10 -n 100
num_iterations = 100
for i in range(1, num_iterations + 1):
    # Select row i (guaranteed to be unique due to shuffling)
    #data = [{"data": globals()[f'sample_{i}']}]
    data = [{"data": [13,13,13,0,13,13,13,13,-76,-76,-81,-76,-78.5,-76,-76,-7,-7,-12,-7,-9.5,-7,-7,12,12,7,12,9.5,12,12]}]
    result = handler.handle(data)
    #print("result: " ,result)

373 ms ± 5.35 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [81]:
%%timeit -r 10 -n 100
num_iterations = 128
for i in range(1, num_iterations + 1):
    # Select row i (guaranteed to be unique due to shuffling)
    #data = [{"data": globals()[f'sample_{i}']}]
    data = [{"data": [13,13,13,0,13,13,13,13,-76,-76,-81,-76,-78.5,-76,-76,-7,-7,-12,-7,-9.5,-7,-7,12,12,7,12,9.5,12,12]}]
    result = handler.handle(data)
    #print("result: " ,result)

478 ms ± 8.02 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [82]:
%%timeit -r 10 -n 100
num_iterations = 256
for i in range(1, num_iterations + 1):
    # Select row i (guaranteed to be unique due to shuffling)
    #data = [{"data": globals()[f'sample_{i}']}]
    data = [{"data": [13,13,13,0,13,13,13,13,-76,-76,-81,-76,-78.5,-76,-76,-7,-7,-12,-7,-9.5,-7,-7,12,12,7,12,9.5,12,12]}]
    result = handler.handle(data)
    #print("result: " ,result)

949 ms ± 9.27 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [83]:
%%timeit -r 10 -n 100
num_iterations = 512
for i in range(1, num_iterations + 1):
    # Select row i (guaranteed to be unique due to shuffling)
    data = [{"data": globals()[f'sample_{i}']}]
    #data = [{"data": [13,13,13,0,13,13,13,13,-76,-76,-81,-76,-78.5,-76,-76,-7,-7,-12,-7,-9.5,-7,-7,12,12,7,12,9.5,12,12]}]
    result = handler.handle(data)
    #print("result: " ,result)

2.08 s ± 13 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [8]:
%%timeit -r 1 -n 1
data = [
      {"data": [13,13,13,0,13,13,13,13,-76,-76,-81,-76,-78.5,-76,-76,-7,-7,-12,-7,-9.5,-7,-7,12,12,7,12,9.5,12,12]},
      {"data": [13,13,12,0.4714045208,13,12.5,13,13,-81,-81,-82,-81,-81.5,-81,-81,-12,-12,-11,-12,-12,-12,-11.5,7,7,2,7,4.5,7,7]},
      {"data": [6,7,7,0.4714045208,7,6.5,7,7,-107,-106,-106,-106,-106.5,-106,-106,-13,-14,-14,-14,-14,-14,-13.5,2,2,2,2,2,2,2]},
      {"data": [8,8,8,0,8,8,8,8,-108,-108,-108,-108,-108,-108,-108,-13,-13,-13,-13,-13,-13,-13,2,2,2,2,2,2,2]}
]

result = handler.handle(data)

print("result: ", result)

result:  ['No Stall', 'No Stall', 'Stall', 'Stall']
2.85 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [9]:
%%timeit -r 1 -n 1

data1 = [{"data": [13,13,13,0,13,13,13,13,-76,-76,-81,-76,-78.5,-76,-76,-7,-7,-12,-7,-9.5,-7,-7,12,12,7,12,9.5,12,12]}]
data2 = [{"data": [13,13,12,0.4714045208,13,12.5,13,13,-81,-81,-82,-81,-81.5,-81,-81,-12,-12,-11,-12,-12,-12,-11.5,7,7,2,7,4.5,7,7]}]
data3 = [{"data": [6,7,7,0.4714045208,7,6.5,7,7,-107,-106,-106,-106,-106.5,-106,-106,-13,-14,-14,-14,-14,-14,-13.5,2,2,2,2,2,2,2]}]
data4 = [{"data": [8,8,8,0,8,8,8,8,-108,-108,-108,-108,-108,-108,-108,-13,-13,-13,-13,-13,-13,-13,2,2,2,2,2,2,2]}]

result1 = handler.handle(data1)
print("result 1:", result1)

result2 = handler.handle(data2)
print("result 2:", result2)

result3 = handler.handle(data3)
print("result 3:", result3)

result4 = handler.handle(data4)
print("result 4:", result4)

result 1: ['No Stall']
result 2: ['No Stall']
result 3: ['Stall']
result 4: ['Stall']
30.1 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [10]:
%%timeit -r 1 -n 1
from concurrent.futures import ThreadPoolExecutor

# Define a task function to handle data
def handle_data(data, identifier):
    handler = MyHandler(model)
    result = handler.handle(data)
    return f"Task {identifier} finished with result: {result}"


# Example datasets
datasets = [
    ([{"data": [13, 13, 13, 0, 13, 13, 13, 13, -76, -76, -81, -76, -78.5, -76, -76, -7, -7, -12, -7, -9.5, -7, -7, 12, 12, 7, 12, 9.5, 12, 12]}], "Data1"),
    ([{"data": [13, 13, 12, 0.4714045208, 13, 12.5, 13, 13, -81, -81, -82, -81, -81.5, -81, -81, -12, -12, -11, -12, -12, -12, -11.5, 7, 7, 2, 7, 4.5, 7, 7]}], "Data2"),
    ([{"data": [6, 7, 7, 0.4714045208, 7, 6.5, 7, 7, -107, -106, -106, -106, -106.5, -106, -106, -13, -14, -14, -14, -14, -14, -13.5, 2, 2, 2, 2, 2, 2, 2]}], "Data3"),
    ([{"data": [8, 8, 8, 0, 8, 8, 8, 8, -108, -108, -108, -108, -108, -108, -108, -13, -13, -13, -13, -13, -13, -13, 2, 2, 2, 2, 2, 2, 2]}], "Data4"),
]

# Run tasks concurrently
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(handle_data, dataset[0], dataset[1]) for dataset in datasets]

    # Collect and print results as they complete
    for future in futures:
        print(future.result())

Task Data1 finished with result: ['No Stall']
Task Data2 finished with result: ['No Stall']
Task Data3 finished with result: ['Stall']
Task Data4 finished with result: ['Stall']
7.56 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# Kserve

In [95]:
import requests
import json

# Define KServe Inference Service details
INGRESS_HOST = "youtubegoes5g.kubeflow-user-example-com.svc.cluster.local"
INGRESS_PORT = "80"  
SERVICE_HOSTNAME = "youtubegoes5g.kubeflow-user-example-com.svc.cluster.local"
MODEL_NAME = "youtubegoes5g"

# Construct the inference URL and headers
URL = f"http://{INGRESS_HOST}:{INGRESS_PORT}/v1/models/{MODEL_NAME}:predict"
HEADERS = {
    "Host": SERVICE_HOSTNAME,
    "Content-Type": "application/json"
}

# Define the sample data payload
def get_data():
    return {
        "instances": [
            {"data": [13, 13, 13, 0, 13, 13, 13, 13, -76, -76, -81, -76, -78.5, -76, -76, -7, -7, -12, -7, -9.5, -7, -7, 12, 12, 7, 12, 9.5, 12, 12]}
        ]
    }

# Function to handle the inference request
def handle_request():
    payload = json.dumps(get_data())
    response = requests.post(URL, headers=HEADERS, data=payload)
    return response.json()

In [ ]:
%%timeit -r 10 -n 100 
num_iterations = 1
for i in range(1, num_iterations + 1):
    data = get_data()  # Dynamically retrieve data for each iteration
    result = handle_request()
    # Optionally print or process result
    #print(f"Result for iteration {i}: {result}")

In [ ]:
%%timeit -r 10 -n 100 
num_iterations = 4
for i in range(1, num_iterations + 1):
    data = get_data()  # Dynamically retrieve data for each iteration
    result = handle_request()
    # Optionally print or process result
    #print(f"Result for iteration {i}: {result}")

In [100]:
%%timeit -r 10 -n 100 
num_iterations = 8
for i in range(1, num_iterations + 1):
    data = get_data()  # Dynamically retrieve data for each iteration
    result = handle_request()
    # Optionally print or process result
    #print(f"Result for iteration {i}: {result}")

158 ms ± 3.24 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [101]:
%%timeit -r 10 -n 100 
num_iterations = 16
for i in range(1, num_iterations + 1):
    data = get_data()  # Dynamically retrieve data for each iteration
    result = handle_request()
    # Optionally print or process result
    #print(f"Result for iteration {i}: {result}")

316 ms ± 4.09 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [102]:
%%timeit -r 10 -n 100 
num_iterations = 32
for i in range(1, num_iterations + 1):
    data = get_data()  # Dynamically retrieve data for each iteration
    result = handle_request()
    # Optionally print or process result
    #print(f"Result for iteration {i}: {result}")

630 ms ± 5.55 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [103]:
%%timeit -r 10 -n 100 
num_iterations = 100
for i in range(1, num_iterations + 1):
    data = get_data()  # Dynamically retrieve data for each iteration
    result = handle_request()
    # Optionally print or process result
    #print(f"Result for iteration {i}: {result}")

1.97 s ± 6.59 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [104]:
%%timeit -r 10 -n 100 
num_iterations = 128
for i in range(1, num_iterations + 1):
    data = get_data()  # Dynamically retrieve data for each iteration
    result = handle_request()
    # Optionally print or process result
    #print(f"Result for iteration {i}: {result}")

2.52 s ± 9.88 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [105]:
%%timeit -r 10 -n 100 
num_iterations = 256
for i in range(1, num_iterations + 1):
    data = get_data()  # Dynamically retrieve data for each iteration
    result = handle_request()
    # Optionally print or process result
    #print(f"Result for iteration {i}: {result}")

5.05 s ± 12.8 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [106]:
%%timeit -r 10 -n 100 
num_iterations = 512
for i in range(1, num_iterations + 1):
    data = get_data()  # Dynamically retrieve data for each iteration
    result = handle_request()
    # Optionally print or process result
    #print(f"Result for iteration {i}: {result}")

10.1 s ± 17.4 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)
